<a href="https://colab.research.google.com/github/annajliu110101/Uber-Driver-Pay-Algorithm/blob/main/notebooks/Exploaration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/annajliu110101/Uber-Driver-Pay-Algorithm.git
!cd /content/Uber-Driver-Pay-Algorithm/

Cloning into 'Uber-Driver-Pay-Algorithm'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 75 (delta 7), reused 64 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (75/75), 12.43 KiB | 3.11 MiB/s, done.
Resolving deltas: 100% (7/7), done.
Filtering content: 100% (22/22), 3.45 GiB | 48.05 MiB/s, done.


In [ ]:
# Uber Fare Prediction – Milestone 2 Exploration

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
files = glob.glob("/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/*.parquet")
print("Found", len(files), "files")
print("First few files:", files[:5])
dfs = [pd.read_parquet(f) for f in files]
df = pd.concat(dfs, ignore_index=True)
df.head()

Found 11 files
First few files: ['/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/fhvhv_tripdata_2024-08.parquet', '/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/fhvhv_tripdata_2025-05.parquet', '/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/fhvhv_tripdata_2025-01.parquet', '/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/fhvhv_tripdata_2025-02.parquet', '/content/Uber-Driver-Pay-Algorithm/data/uber_nyc_forhire_2025/fhvhv_tripdata_2024-09.parquet']
